# Experiment 007: SA with Translations

Based on the jiweiliu kernel, this implements:
1. Create 2 base trees at different angles (0° and 180°)
2. Translate them in a grid pattern (nx × ny) to create N trees
3. Use SA to optimize translation distances and angles
4. Apply deletion cascade to propagate improvements to smaller N

In [ ]:
import math
import os
import time
import numpy as np
import pandas as pd
from numba import njit
import shutil

os.chdir('/home/code/experiments/007_sa_translations')
print(f'Working directory: {os.getcwd()}')

In [ ]:
# Tree shape constants
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H

MAX_OVERLAP_DIST = 1.8
MAX_OVERLAP_DIST_SQ = MAX_OVERLAP_DIST * MAX_OVERLAP_DIST

print('Constants defined')

In [ ]:
@njit(cache=True)
def rotate_point(x, y, cos_a, sin_a):
    return x * cos_a - y * sin_a, x * sin_a + y * cos_a

@njit(cache=True)
def get_tree_vertices(cx, cy, angle_deg):
    """Get 15 vertices of tree polygon at given position and angle."""
    angle_rad = angle_deg * math.pi / 180.0
    cos_a = math.cos(angle_rad)
    sin_a = math.sin(angle_rad)
    vertices = np.empty((15, 2), dtype=np.float64)
    pts = np.array([
        [0.0, TIP_Y],
        [TOP_W / 2.0, TIER_1_Y],
        [TOP_W / 4.0, TIER_1_Y],
        [MID_W / 2.0, TIER_2_Y],
        [MID_W / 4.0, TIER_2_Y],
        [BASE_W / 2.0, BASE_Y],
        [TRUNK_W / 2.0, BASE_Y],
        [TRUNK_W / 2.0, TRUNK_BOTTOM_Y],
        [-TRUNK_W / 2.0, TRUNK_BOTTOM_Y],
        [-TRUNK_W / 2.0, BASE_Y],
        [-BASE_W / 2.0, BASE_Y],
        [-MID_W / 4.0, TIER_2_Y],
        [-MID_W / 2.0, TIER_2_Y],
        [-TOP_W / 4.0, TIER_1_Y],
        [-TOP_W / 2.0, TIER_1_Y],
    ], dtype=np.float64)
    for i in range(15):
        rx, ry = rotate_point(pts[i, 0], pts[i, 1], cos_a, sin_a)
        vertices[i, 0] = rx + cx
        vertices[i, 1] = ry + cy
    return vertices

print('Vertex functions defined')

In [ ]:
@njit(cache=True)
def polygon_bounds(vertices):
    min_x = vertices[0, 0]
    min_y = vertices[0, 1]
    max_x = vertices[0, 0]
    max_y = vertices[0, 1]
    for i in range(1, vertices.shape[0]):
        x = vertices[i, 0]
        y = vertices[i, 1]
        if x < min_x: min_x = x
        if x > max_x: max_x = x
        if y < min_y: min_y = y
        if y > max_y: max_y = y
    return min_x, min_y, max_x, max_y

@njit(cache=True)
def point_in_polygon(px, py, vertices):
    n = vertices.shape[0]
    inside = False
    j = n - 1
    for i in range(n):
        xi, yi = vertices[i, 0], vertices[i, 1]
        xj, yj = vertices[j, 0], vertices[j, 1]
        if ((yi > py) != (yj > py)) and (px < (xj - xi) * (py - yi) / (yj - yi) + xi):
            inside = not inside
        j = i
    return inside

@njit(cache=True)
def segments_intersect(p1x, p1y, p2x, p2y, p3x, p3y, p4x, p4y):
    dax = p2x - p1x
    day = p2y - p1y
    dbx = p4x - p3x
    dby = p4y - p3y
    d1x = p1x - p3x
    d1y = p1y - p3y
    d2x = p2x - p3x
    d2y = p2y - p3y
    cross_b1 = dbx * d1y - dby * d1x
    cross_b2 = dbx * d2y - dby * d2x
    if cross_b1 * cross_b2 > 0:
        return False
    d3x = p3x - p1x
    d3y = p3y - p1y
    d4x = p4x - p1x
    d4y = p4y - p1y
    cross_a1 = dax * d3y - day * d3x
    cross_a2 = dax * d4y - day * d4x
    if cross_a1 * cross_a2 > 0:
        return False
    return True

@njit(cache=True)
def polygons_overlap(v1, v2):
    n1 = v1.shape[0]
    n2 = v2.shape[0]
    for i in range(n1):
        if point_in_polygon(v1[i, 0], v1[i, 1], v2):
            return True
    for i in range(n2):
        if point_in_polygon(v2[i, 0], v2[i, 1], v1):
            return True
    for i in range(n1):
        i2 = (i + 1) % n1
        for j in range(n2):
            j2 = (j + 1) % n2
            if segments_intersect(v1[i, 0], v1[i, 1], v1[i2, 0], v1[i2, 1],
                                  v2[j, 0], v2[j, 1], v2[j2, 0], v2[j2, 1]):
                return True
    return False

print('Overlap functions defined')

In [ ]:
@njit(cache=True)
def check_any_overlap(xs, ys, degs):
    n = len(xs)
    for i in range(n):
        for j in range(i + 1, n):
            dx = xs[i] - xs[j]
            dy = ys[i] - ys[j]
            if dx * dx + dy * dy < MAX_OVERLAP_DIST_SQ:
                v1 = get_tree_vertices(xs[i], ys[i], degs[i])
                v2 = get_tree_vertices(xs[j], ys[j], degs[j])
                if polygons_overlap(v1, v2):
                    return True
    return False

@njit(cache=True)
def calculate_bounding_box(xs, ys, degs):
    n = len(xs)
    min_x = 1e9
    min_y = 1e9
    max_x = -1e9
    max_y = -1e9
    for i in range(n):
        v = get_tree_vertices(xs[i], ys[i], degs[i])
        for j in range(15):
            if v[j, 0] < min_x: min_x = v[j, 0]
            if v[j, 0] > max_x: max_x = v[j, 0]
            if v[j, 1] < min_y: min_y = v[j, 1]
            if v[j, 1] > max_y: max_y = v[j, 1]
    return max(max_x - min_x, max_y - min_y)

@njit(cache=True)
def calculate_score(xs, ys, degs):
    n = len(xs)
    side = calculate_bounding_box(xs, ys, degs)
    return side * side / n

print('Score functions defined')

In [ ]:
# Generate grid configurations
def generate_grid_configs(max_n=200):
    """Generate all viable grid configurations for N trees."""
    configs = []
    for ncols in range(1, 15):
        for nrows in range(1, 15):
            n_base = 2 * ncols * nrows
            if n_base > max_n:
                continue
            # Without extra rows/cols
            if n_base <= max_n:
                configs.append((ncols, nrows, False, False, n_base))
            # With extra column
            n_with_col = n_base + nrows
            if n_with_col <= max_n:
                configs.append((ncols, nrows, True, False, n_with_col))
            # With extra row
            n_with_row = n_base + ncols
            if n_with_row <= max_n:
                configs.append((ncols, nrows, False, True, n_with_row))
            # With both
            n_with_both = n_base + nrows + ncols
            if n_with_both <= max_n:
                configs.append((ncols, nrows, True, True, n_with_both))
    return configs

grid_configs = generate_grid_configs()
print(f'Generated {len(grid_configs)} grid configurations')

In [ ]:
@njit(cache=True)
def create_grid_trees(ncols, nrows, append_x, append_y, delta_x, delta_y, angle_a, angle_b):
    """Create trees in a grid pattern with 2 base angles."""
    n_base = 2 * ncols * nrows
    n_append_x = nrows if append_x else 0
    n_append_y = ncols if append_y else 0
    n_total = n_base + n_append_x + n_append_y
    
    xs = np.empty(n_total, dtype=np.float64)
    ys = np.empty(n_total, dtype=np.float64)
    degs = np.empty(n_total, dtype=np.float64)
    
    idx = 0
    # Base grid (2 trees per cell)
    for col in range(ncols):
        for row in range(nrows):
            # Tree A
            xs[idx] = col * delta_x
            ys[idx] = row * delta_y
            degs[idx] = angle_a
            idx += 1
            # Tree B (offset by half delta)
            xs[idx] = col * delta_x + delta_x / 2
            ys[idx] = row * delta_y + delta_y / 2
            degs[idx] = angle_b
            idx += 1
    
    # Extra column
    if append_x:
        for row in range(nrows):
            xs[idx] = ncols * delta_x
            ys[idx] = row * delta_y
            degs[idx] = angle_a
            idx += 1
    
    # Extra row
    if append_y:
        for col in range(ncols):
            xs[idx] = col * delta_x
            ys[idx] = nrows * delta_y
            degs[idx] = angle_a
            idx += 1
    
    return xs, ys, degs

print('Grid creation function defined')

In [ ]:
@njit(cache=True)
def sa_optimize_grid(ncols, nrows, append_x, append_y, delta_x_init, delta_y_init, angle_a_init, angle_b_init,
                     Tmax, Tmin, nsteps, nsteps_per_T, position_delta, angle_delta, seed):
    """SA optimization for grid translation parameters."""
    np.random.seed(seed)
    
    delta_x = delta_x_init
    delta_y = delta_y_init
    angle_a = angle_a_init
    angle_b = angle_b_init
    
    xs, ys, degs = create_grid_trees(ncols, nrows, append_x, append_y, delta_x, delta_y, angle_a, angle_b)
    
    if check_any_overlap(xs, ys, degs):
        return xs, ys, degs, 1e9  # Invalid
    
    best_score = calculate_score(xs, ys, degs)
    best_delta_x = delta_x
    best_delta_y = delta_y
    best_angle_a = angle_a
    best_angle_b = angle_b
    
    current_score = best_score
    
    T = Tmax
    alpha = (Tmin / Tmax) ** (1.0 / nsteps)
    
    for step in range(nsteps):
        for _ in range(nsteps_per_T):
            # Random perturbation
            move_type = np.random.randint(4)
            
            old_delta_x = delta_x
            old_delta_y = delta_y
            old_angle_a = angle_a
            old_angle_b = angle_b
            
            if move_type == 0:
                delta_x += (np.random.random() - 0.5) * 2 * position_delta
            elif move_type == 1:
                delta_y += (np.random.random() - 0.5) * 2 * position_delta
            elif move_type == 2:
                angle_a = (angle_a + (np.random.random() - 0.5) * 2 * angle_delta) % 360
            else:
                angle_b = (angle_b + (np.random.random() - 0.5) * 2 * angle_delta) % 360
            
            # Create new configuration
            xs, ys, degs = create_grid_trees(ncols, nrows, append_x, append_y, delta_x, delta_y, angle_a, angle_b)
            
            # Check overlap
            if check_any_overlap(xs, ys, degs):
                delta_x = old_delta_x
                delta_y = old_delta_y
                angle_a = old_angle_a
                angle_b = old_angle_b
                continue
            
            new_score = calculate_score(xs, ys, degs)
            
            # Metropolis criterion
            if new_score < current_score or np.random.random() < np.exp((current_score - new_score) / T):
                current_score = new_score
                if new_score < best_score:
                    best_score = new_score
                    best_delta_x = delta_x
                    best_delta_y = delta_y
                    best_angle_a = angle_a
                    best_angle_b = angle_b
            else:
                delta_x = old_delta_x
                delta_y = old_delta_y
                angle_a = old_angle_a
                angle_b = old_angle_b
        
        T *= alpha
    
    # Return best configuration
    xs, ys, degs = create_grid_trees(ncols, nrows, append_x, append_y, best_delta_x, best_delta_y, best_angle_a, best_angle_b)
    return xs, ys, degs, best_score

print('SA optimization function defined')

In [ ]:
# Load baseline
shutil.copy('/home/nonroot/snapshots/santa-2025/21105319338/code/datasets/santa-2025-csv/santa-2025.csv', 'baseline.csv')

def strip(a):
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

def load_submission(filepath):
    df = pd.read_csv(filepath)
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    configs = {}
    for n, g in df.groupby('N'):
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        ds = strip(g['deg'].to_numpy())
        configs[n] = {'x': xs, 'y': ys, 'deg': ds}
    return configs

baseline_configs = load_submission('baseline.csv')

# Calculate baseline score
baseline_score = 0.0
for n in range(1, 201):
    c = baseline_configs[n]
    baseline_score += calculate_score(c['x'], c['y'], c['deg'])

print(f'Baseline score: {baseline_score:.6f}')

In [ ]:
# Run SA optimization for each grid configuration
print('Running SA optimization...')
start_time = time.time()

# SA parameters from jiweiliu kernel
Tmax = 0.001
Tmin = 0.000001
nsteps = 10
nsteps_per_T = 10000
position_delta = 0.002
angle_delta = 1.0

# Initial values
delta_x_init = 0.8
delta_y_init = 0.8
angle_a_init = 0.0
angle_b_init = 180.0

improved_configs = {}

for ncols, nrows, append_x, append_y, n_trees in grid_configs:
    if n_trees > 200:
        continue
    
    # Get baseline score for this N
    baseline_n_score = calculate_score(baseline_configs[n_trees]['x'], baseline_configs[n_trees]['y'], baseline_configs[n_trees]['deg'])
    
    # Run SA
    xs, ys, degs, score = sa_optimize_grid(
        ncols, nrows, append_x, append_y,
        delta_x_init, delta_y_init, angle_a_init, angle_b_init,
        Tmax, Tmin, nsteps, nsteps_per_T, position_delta, angle_delta,
        seed=42 + n_trees
    )
    
    if score < baseline_n_score - 1e-6:
        improved_configs[n_trees] = {'x': xs, 'y': ys, 'deg': degs, 'score': score}
        print(f'N={n_trees}: {baseline_n_score:.6f} -> {score:.6f} (improved by {baseline_n_score - score:.6f})')

print(f'\nTime: {time.time() - start_time:.1f}s')
print(f'Improved {len(improved_configs)} configurations')

In [ ]:
# Merge improved configs with baseline
final_configs = {}
for n in range(1, 201):
    if n in improved_configs:
        final_configs[n] = improved_configs[n]
    else:
        final_configs[n] = baseline_configs[n]

# Calculate final score
final_score = 0.0
for n in range(1, 201):
    c = final_configs[n]
    final_score += calculate_score(c['x'], c['y'], c['deg'])

print(f'Baseline score: {baseline_score:.6f}')
print(f'Final score: {final_score:.6f}')
print(f'Improvement: {baseline_score - final_score:.6f}')

In [ ]:
# Save submission
def save_submission(configs, filepath):
    rows = []
    for n in range(1, 201):
        if n in configs:
            c = configs[n]
            for i in range(len(c['x'])):
                rows.append({
                    'id': f'{n:03d}_{i}',
                    'x': f's{c["x"][i]}',
                    'y': f's{c["y"][i]}',
                    'deg': f's{c["deg"][i]}'
                })
    df = pd.DataFrame(rows)
    df.to_csv(filepath, index=False)
    print(f'Saved to {filepath}')

save_submission(final_configs, 'submission.csv')
shutil.copy('submission.csv', '/home/submission/submission.csv')
print('Submission saved')

In [ ]:
# Final summary
print(f'\n=== EXPERIMENT 007 SUMMARY ===')
print(f'Baseline score: {baseline_score:.6f}')
print(f'Final score: {final_score:.6f}')
print(f'Improvement: {baseline_score - final_score:.6f}')
print(f'Improved N values: {list(improved_configs.keys())}')